In [1]:
import os
import scipy.io
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader
from tqdm import tqdm

from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torchvision import transforms

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from modules import NormedConv2d,BcosConv2d,NormedLinear
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

import tifffile as tiff
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import tifffile as tiff
import numpy as np

import os
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import torchvision.transforms as transforms


In [3]:
if torch.cuda.is_available():
	device = torch.device("cuda")
elif torch.backends.mps.is_available():
	device = torch.device("mps")
else:
	device = torch.device("cpu")
print (device)

mps


In [4]:
root_path = './processed_female_cells/'
files = os.listdir(root_path)
files = [i for i in files if not i== '.DS_Store']
file_labels = [i.split(' ')[2] for i in files]
label_string_to_numreic = {'Mutant':0, 'Control':1, 'Treated':2}
labels = [label_string_to_numreic[i] for i in file_labels]
train_files, test_files, train_labels, test_labels = train_test_split(files,labels, test_size=0.2, random_state=42, stratify=labels)

image_paths_train = []
labels_train = []
for i,ele in enumerate(train_files):
    image_path = os.listdir(root_path+ele)
    image_path = [k for k in image_path if k.endswith('.png')]
    for k in image_path:
        image_paths_train.append(root_path+ele+'/'+k)
        labels_train.append(train_labels[i])

image_paths_test = []
labels_test = []
for i,ele in enumerate(test_files):
    image_path = os.listdir(root_path+ele)
    image_path = [k for k in image_path if k.endswith('.png')]
    for k in image_path:
        image_paths_test.append(root_path+ele+'/'+k)
        labels_test.append(test_labels[i])


In [5]:
print ([np.shape(i) for i in [image_paths_train, labels_train, image_paths_test, labels_test]])
print (np.unique(labels_train,return_counts=True),np.unique(labels_test,return_counts=True))

[(27000,), (27000,), (7500,), (7500,)]
(array([0, 1, 2]), array([9000, 9000, 9000])) (array([0, 1, 2]), array([1500, 3000, 3000]))


In [6]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)        
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label

In [7]:
model = torch.hub.load('B-cos/B-cos-v2', 'vitc_l_patch1_14', pretrained=True)
model[0].linear_head.linear.linear = NormedLinear(1024,2)
model = model.to(device)

Using cache found in /Users/helium/.cache/torch/hub/B-cos_B-cos-v2_main


In [8]:
transform = model.transform #transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor()])

train_dataset = CustomDataset(image_paths_train, labels_train, transform=transform)
test_dataset = CustomDataset(image_paths_test, labels_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
for images, labels in train_loader:
    print (images.shape,labels.shape)
    break

torch.Size([32, 6, 224, 224]) torch.Size([32])


In [10]:
model.train()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
softmax = nn.Softmax()

In [11]:
state_dict = torch.load('./model_2.pth',map_location=torch.device(device))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [12]:
def calculate_accuracy(model, data_loader=test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)  # labels are now integers
            outputs = model(inputs)
            # No activation needed; directly take the argmax of the logits
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total * 100.0
    return accuracy
print (calculate_accuracy(model))

100%|█████████████████████████████████████████| 235/235 [03:15<00:00,  1.20it/s]

60.0


In [13]:
epochs = 500
test_log = 0
log = []
counter = 0
for epoch in range(epochs):
    counter +=1
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc="Training"):
        model.train()
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        #outputs = sigmoid(outputs.to(device))
        labels = labels.to(device)
        loss = criterion(outputs.to(device), labels)
        loss.backward()
        optimizer.step()
    test_acc = calculate_accuracy(model,test_loader)
    print (test_acc)
    log.append(test_acc)
    if test_acc>test_log:
        counter = 0
        test_log = test_acc
        #torch.save(model.state_dict(), './model_2.pth')
    if counter > 40:
        break


In [14]:
test_acc = calculate_accuracy(model,test_loader)
print (test_acc)